    Import các thư viện cần thiết

In [21]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from tqdm import tqdm
import csv
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
import pandas as pd
import unidecode
from cmath import nan
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler , RobustScaler, StandardScaler
import warnings
import re
import threading
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import os
warnings.filterwarnings("ignore")

    Crawl data

    URL = https://dathanhdo.com/nha-ban.html

Khởi tạo đối tượng webdriver và khai báo link URL

In [22]:
#URL = https://dathanhdo.com/nha-ban.html
productLinks = []

Lấy tất cả các link của bất động sản tại Đà Nẵng

In [23]:
urlDaNang = "https://dathanhdo.com/nha-ban.html?page{}"
browser = webdriver.Chrome(executable_path="./chromedriver.exe")
for pagenum in range(1,54):
   try:
      browser.get(urlDaNang.format(pagenum))
      tagAs = browser.find_elements(By.CLASS_NAME, "bds-name")
      for tagA in tagAs:
         productLinks.append(tagA.find_element(By.TAG_NAME, "a").get_attribute("href"))
   except:
      continue
productLinks

Lấy các thuộc tính của bất động sản

In [ ]:
detailProductsDN = []
for link in productLinks:
    try:
        browser.get(link)
        try:
            name = browser.find_element(By.CLASS_NAME, "main-name").find_element(By.TAG_NAME, "h1").text
            detailProduct = {"name": name}
        except:
            pass
        try:
            trend = browser.find_element(By.CLASS_NAME, "bds-string").find_element(By.CLASS_NAME, "trend").text
            detailProduct[trend.split(":")[0]] = trend.split(":")[1]
        except:
            pass
        try:
            address = browser.find_element(By.CLASS_NAME, "bds-string").find_element(By.CLASS_NAME, "address").text
            detailProduct["Địa chỉ"] = address.split(":")[1]
        except:
            pass
        try:
            floor = browser.find_element(By.CLASS_NAME, "bds-string").find_element(By.CLASS_NAME, "floor").text
            detailProduct[floor.split(":")[0]] = floor.split(":")[1]
        except:
            pass
        try:
            room = browser.find_element(By.CLASS_NAME, "bds-string").find_element(By.CLASS_NAME, "room").text
            detailProduct[room.split(":")[0]] = room.split(":")[1]
        except:
            pass
        try:
            juridical = browser.find_element(By.CLASS_NAME, "bds-string").find_element(By.TAG_NAME, "juridical").text
            detailProduct[juridical.split(":")[0]]= juridical.split(":")[1]
        except:
            pass
        try:
            titleValues = browser.find_elements(By.CLASS_NAME, "notitle")
            for tv in titleValues:
                try:
                    detailProduct[tv.text.split("\n")[0]]= tv.text.split("\n")[1]
                except:
                    pass
        except:
            pass
        detailProductsDN.append(detailProduct)
    except:
        print("link")
        continue
browser.quit()
detailProductsDN

[{'name': 'BÁN ĐẤT ĐƯỜNG LÊ QUẢNG CHÍ, ĐƯỜNG THÔNG, VỊ TRÍ KINH DOANH BUÔN BÁN',
  'Hướng': ' Tây Bắc',
  'Địa chỉ': ' lê quảng chí, Cẩm Lệ, Đà Nẵng',
  'Mức giá': '7.1 tỷ',
  'Diện tích': '100m2',
  'Lượt xem': '171'},
 {'name': '3 LÔ LIỀN KỀ GẦN CẦU HÒA XUÂN - ĐƯỜNG TRUNG LƯƠNG 18',
  'Hướng': ' Tây Nam',
  'Địa chỉ': ' Trung Lương 18, Cẩm Lệ, Đà Nẵng',
  'Mức giá': '16.8 tỷ',
  'Diện tích': '360m2',
  'Lượt xem': '176'},
 {'name': 'BÁN LÔ ĐẤT ĐÔNG NAM - MẶT TIỀN MAI CHÍ THỌ - KHU ĐẢO VIP SIÊU ĐẸP',
  'Hướng': ' Đông Nam',
  'Địa chỉ': ' MAI CHÍ THỌ, Cẩm Lệ, Đà Nẵng',
  'Mức giá': '5.3 tỷ',
  'Diện tích': '100m2',
  'Lượt xem': '169'},
 {'name': 'BÁN BIỆT THỰ ĐƯỜNG CHẾ VIẾT TẤN VIEW KÊNH THOÁNG MÁT - HÒA XUÂN',
  'Hướng': ' Tây Bắc',
  'Địa chỉ': ' CHẾ VIẾT TẤN, Cẩm Lệ, Đà Nẵng',
  'Mức giá': '13.8 tỷ',
  'Diện tích': '250m2',
  'Lượt xem': '171'},
 {'name': 'CHÀO BÁN SIÊU PHẨM BT 3 TẦNG ĐƯỜNG LÊ QUANG HOÀ - HOÀ XUÂN - TP ĐÀ NẴNG',
  'Hướng': ' Tây Nam',
  'Địa chỉ': ' LÊ QUANG HÒA, 

Ghi dữ liệu qua file excel

In [ ]:
csv_columns = []
for product in detailProductsDN:
    tempkhac = set(list(product)[0:]) ^ set(csv_columns)
    tempgiong = set(list(product)[0:]) & set(csv_columns)
    csv_columns = tempkhac ^ tempgiong
csv_file = "dataBatDongSanDNThanhDo.csv"
try:
    with open(csv_file, 'w+',encoding= "utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for dataz in detailProductsDN:
            try:
                writer.writerow(dataz)
            except:
                continue
except IOError:
    print("I/O error")